In [1]:
# %pip install dill
# %pip install nltk
# %pip install inflect
# %pip install pydantic

In [2]:
import sys
import openai

sys.path.insert(0, "../")
sys.path.insert(0, "../..")
from GenAgentsBoardroom.text_adventure_games.parsing import GptParser3
from GenAgentsBoardroom.test.game_setup import build_boardroom

game = build_boardroom(
    experiment_name="boardroom-test",
    experiment_id=2,
    num_characters=3,
    max_ticks=4,
    personas_path="game_personas",
    # random_placement=True,
)
game.give_hints = True
parser = GptParser3(game, verbose=False)
game.set_parser(parser)
parser.refresh_command_list()

try:
    game.game_loop()
except openai.APIConnectionError:
    print("Could not connect to OpenAI API")
finally:
    game.save_simulation_data()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kylesullivan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kylesullivan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


CHARACTER JSONS: []

GET OR CREATE BASE FACTS PLAYER DESCRIPTION: An quirky contestant that is must see TV on a reality show. True
RESPONSE: Name='Jasper Luv' Age=28 Likes=['drama', 'improv comedy', 'clowning services', 'unconventional fashion'] Dislikes=['predictability', 'silence', 'seriousness'] Occupation='Reality TV Contestant' Home_City='Los Angeles, California'
Generated facts: {'Name': 'Jasper Luv', 'Age': 28, 'Likes': ['drama', 'improv comedy', 'clowning services', 'unconventional fashion'], 'Dislikes': ['predictability', 'silence', 'seriousness'], 'Occupation': 'Reality TV Contestant', 'Home_City': 'Los Angeles, California'}
PERSONA: Jasper Luv, a 28-year-old Reality TV Contestant. You are passionate about drama, improv comedy, clowning services. You have aversions to predictability, silence, seriousness. Some key facts about yourself are: home_city, Los Angeles, California. Your overall game strategy is Tit-for-Tat, reflecting your judgement which *tends* to be Mostly, coope

KeyboardInterrupt: 

In [20]:
import json
from GenAgentsBoardroom.text_adventure_games.parsing import Parser

experiment_id = "1"
experiment_name = "boardroom-test"

wrap_width = 120

def print_with_symbols(ratio, total_length, message, symbol):
    message_length = len(message) + 2  # Including the surrounding symbols
    num_symbols = (total_length - message_length) // (ratio + 1)  # Calculate number of symbols
    num_spaces = (total_length - (num_symbols * 2 + message_length)) // 2  # Calculate number of spaces
    print(f"{' ' * num_spaces}{symbol * num_symbols} {message} {symbol * num_symbols}\n")

# Read the JSONL file and print the 'message' field
with open(
    f"/Users/kylesullivan/Documents/Professional/LLM Agents/GenAgentsBoardroom/GenAgentsBoardroom/logs/{experiment_name}-{experiment_id}/gpt_call_{experiment_name}-{experiment_id}.jsonl",
    "r",
) as file:
    for i, line in enumerate(file):
        data = json.loads(line)
        message = data.get("messages", "No message found")
        character = data.get("character_name", "No character found")
        round = data.get("round", "No round found")
        tick = data.get("tick", "No tick found")

        # Calculate the number of symbols needed for formatting
        title = f"{character}: {round}.{tick}"
        symbols_needed = (wrap_width - len(title) - 2) // 2
        symbols = "=" * symbols_needed

        print(f"\n{symbols} {title} {symbols}\n")

        # Check if message is a list
        if isinstance(message, list):
            for msg in message:
                if isinstance(msg, tuple | list) and len(msg) == 2:
                    total_length = 120
                    ratio = 2
                    print_with_symbols(ratio, total_length, msg[0], '~')
                    print(Parser.wrap_text(msg[1]), "\n")
                else:
                    print("FAILURE:", type(msg), Parser.wrap_text(msg))
        else:
            print(Parser.wrap_text(message))

        print_with_symbols(ratio, total_length, "Response", "~")
        response_data = data.get("response", "No response found")

        # Check if response_data is a valid JSON string before loading
        if response_data and isinstance(response_data, str):
            try:
                response_json = json.loads(response_data)
                if isinstance(response_json, dict):
                    for key, value in response_json.items():
                        print(Parser.wrap_text(f"{key}:"), end="\n")
                        if (
                            isinstance(value, list)
                            and len(value) > 0
                            and isinstance(value[0], dict)
                        ):
                            for item in value:
                                for k, v in item.items():
                                    print(Parser.wrap_text(f"{k}:\n- {str(v)}\n"))
                        else:
                            print(Parser.wrap_text(f"- {str(value)}\n"))
                else:
                    print(Parser.wrap_text("Response is not a dictionary."))
            except json.JSONDecodeError:
                print(Parser.wrap_text("Error decoding JSON response."))
                print(Parser.wrap_text(response_data))
        else:
            print(Parser.wrap_text("No valid response data found."))


================================================= Blaire Jackson: 0.0 =================================================

                   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ System ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

WORLD INFO:
You are seated at a boardroom meeting for Apple Inc. Q2 2025 Board Meeting, discussing Strategic Planning for the Next
Quarter and Review of Q1 2025 Results. Also present are: Jasper Wild and Jasper Blue. Having carefully read through the
provided documents, you prepare talking points, arguments, and counterarguments for the meeting. Once the meeting
starts, you will be given a chance to speak. Your goal is to present your case and sway the other board members to your
side.

PERSONAL INFO:
You are Blaire Jackson, a 28-year-old Reality Show Contestant. You are passionate about improv comedy, ancient history,
pets. You have aversions to sameness, promotional events, being told what to do. Some key facts about yourself are:
home_city, Austin, Texas. Your o

1.) Go through and figure out where the Goals, Impressions, Memories, etc. should live (system prompt or user prompt).
2.) Convert the dialogue into messages (assistant and user) – maybe store these in dicts for each character?
3.) Set up generate calls to respond in a specified format. Also include:
    a.) Want to Respond (boolean)
    b.) Response (string)
        i.) Split up the response into multiple pieces/queries (List[string])
        ii.) Summarize the important pieces (List[string])
        iii.) Get the importance scores for each important piece (List[float])
4.) Go through and figure out how to best handle the dialogue history. It needs to be summarized every so often and the
most recent messages need to be shown verbatim. (Optional to do another layer of summarization on the history)
5.) Split up responses into multiple queries to facilitate more accurate cosine similarity scores.
6.) Save each dialogue response in the speaker's memory stream as MemoryType.RESPONSE
7.) Add the persona information to the memory stream as MemoryType.PERSONA
8.) What else???

In [4]:
import numpy as np

In [5]:
relevances = np.array(
    [
        [0.15751655, 0.29064174],
        [0.30757024, 0.37188991],
        [0.36602292, 0.38734758],
        [0.18724868, 0.28652711],
        [0.42884286, 0.44468825],
        [0.31736738, 0.38197477],
        [0.39542971, 0.44803867],
        [0.39302259, 0.47098665],
    ]
)

relevances.shape

(8, 2)

In [6]:
relevances = np.array([[0.15751655, 0.29064174, 0.24367228, 0.13777956, 0.30159348, 0.17114509],
 [0.30757024, 0.37188991, 0.43831708, 0.30076554, 0.43010477, 0.32824591],
 [0.36602292, 0.38734758, 0.41344679, 0.38940488, 0.34685532, 0.33309566],
 [0.18724868, 0.28652711, 0.2390725, 0.1446917, 0.29754809, 0.17871562],
 [0.42884286, 0.44468825, 0.38975823, 0.40420782, 0.36539914, 0.38626809],
 [0.31736738, 0.38197477, 0.35364001, 0.33578253, 0.30384432, 0.30996707],
 [0.39542971, 0.44803867, 0.3852455, 0.32855933, 0.37437779, 0.42213412],
 [0.39302259, 0.47098665, 0.38014296, 0.32390701, 0.40132384, 0.269162  ]])

threshold = np.array([0.29064174, 0.43010477, 0.38940488, 0.28652711, 0.42884286, 0.35364001, 0.42213412, 0.40132384])

# Filter the relevances to only include scores above the threshold
filtered_scores = np.array(
    [
        relevances[i][relevances[i] >= threshold[i]]
        for i in range(relevances.shape[0])
    ]
)

filtered_scores

array([[0.29064174, 0.30159348],
       [0.43831708, 0.43010477],
       [0.41344679, 0.38940488],
       [0.28652711, 0.29754809],
       [0.42884286, 0.44468825],
       [0.38197477, 0.35364001],
       [0.44803867, 0.42213412],
       [0.47098665, 0.40132384]])

In [7]:
filtered_indices = np.array(
    [np.where(relevances[i] >= threshold[i])[0] for i in range(relevances.shape[0])]
)

filtered_indices

array([[1, 4],
       [2, 4],
       [2, 3],
       [1, 4],
       [0, 1],
       [1, 2],
       [1, 5],
       [1, 4]])

In [8]:
weights = [(0.0, 5.0), (0.0, 4.0), (0.0, 4.5), (0.0, 3.5), (0.0, 4.0), (0.0, 3.5)]

# Multiply the filtered scores by their respective weights
weighted_scores = []
for i, indices in enumerate(filtered_indices):
    row_weights = [weights[idx] for idx in indices]
    row_scores = filtered_scores[i]
    if isinstance(row_weights[0], tuple):
        weighted_row = [
            tuple(score * w for w in weight) for score, weight in zip(row_scores, row_weights)
        ]
    else:
        weighted_row = [score * weight for score, weight in zip(row_scores, row_weights)]
    weighted_scores.append(weighted_row)

all(isinstance(score, tuple) for row in weighted_scores for score in row)
weighted_scores

[[(0.0, 1.16256696), (0.0, 1.20637392)],
 [(0.0, 1.9724268600000001), (0.0, 1.72041908)],
 [(0.0, 1.860510555), (0.0, 1.3629170800000001)],
 [(0.0, 1.14610844), (0.0, 1.19019236)],
 [(0.0, 2.1442143), (0.0, 1.778753)],
 [(0.0, 1.52789908), (0.0, 1.591380045)],
 [(0.0, 1.79215468), (0.0, 1.47746942)],
 [(0.0, 1.8839466), (0.0, 1.60529536)]]

In [9]:
result = np.array([np.mean(scores, axis=0) for scores in weighted_scores])
result

array([[0.        , 1.18447044],
       [0.        , 1.84642297],
       [0.        , 1.61171382],
       [0.        , 1.1681504 ],
       [0.        , 1.96148365],
       [0.        , 1.55963956],
       [0.        , 1.63481205],
       [0.        , 1.74462098]])

In [10]:
for pair in [(0.521686234197001, (0.0, 4.0)), (0.40867998651182436, (0.0, 3.5)), (0.45331416268939656, (0.0, 4.5)), (0.49028910309500023, (0.0, 3.0)), (0.3382244109438404, (0.0, 5.0))]:
    print(pair[0] * pair[1][0], pair[0] * pair[1][1])

0.0 2.086744936788004
0.0 1.4303799527913852
0.0 2.0399137321022844
0.0 1.4708673092850006
0.0 1.691122054719202


In [11]:
temp = np.array([[4.2, 0.382448], [0.0441, 0.6945525], [0.0, 0.63637325], [0.0, 0.39437435]])
temp.ndim

2

In [12]:
np.sum(temp, axis=1)

array([4.582448  , 0.7386525 , 0.63637325, 0.39437435])

In [13]:
np.sum([4], axis=0)

4

In [14]:
np.sum([(4, 5), (6, 7), (8, 9)], axis=1)


array([ 9, 13, 17])

In [15]:
np.sum([-0.4581541090448713, -0.4490072636711296, -0.14119390089781322, 0.28261750562932914, 0.16525130063244528, 0.42930724670480225])


-0.17117922064723723

In [16]:
np.sum(
    [
        -0.7349755913180069,
        -0.265002419525306,
        -0.11724999255938251,
        0.3223845489833902,
        0.21932538492545783,
        0.28270654157225367,
        0.4639907485688312,
    ]
)

0.17117922064723756

In [17]:
list1 = [1, 2, 3, 4, 5]
tuple1 = (10, 20, 30, 40, 50)

zip1 = zip(list1, tuple1)

zip2 = sorted(zip1, key=lambda x: x[1])

print("type(zip1):", type(zip1))
print("type(zip2):", type(zip2))

type(zip1): <class 'zip'>
type(zip2): <class 'list'>


In [18]:
my_tuple = ((0, 0), (1, 10), (2, 20), (3, 30), (4, 40), (5, 50), (6, 60), (7, 70), (8, 80), (9, 90))
my_tuple = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

# Convert tuple to a numpy array for easier manipulation
data = np.array(my_tuple)

# Calculate mean and std for each index separately
means = np.mean(data, axis=0)
stds = np.std(data, axis=0)

# Normalize each index separately
normalized_data = (data - means) / stds

print("Original Data:\n", my_tuple)
print("Means:\n", means)
print("Standard Deviations:\n", stds)
print("Normalized Data:\n", normalized_data)

Original Data:
 (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
Means:
 4.5
Standard Deviations:
 2.8722813232690143
Normalized Data:
 [-1.5666989  -1.21854359 -0.87038828 -0.52223297 -0.17407766  0.17407766
  0.52223297  0.87038828  1.21854359  1.5666989 ]


In [19]:
(my_list - np.mean(my_list)) / np.std(my_list)

NameError: name 'my_list' is not defined